In [13]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [14]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_fits.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

# c : colour paramter, + means redder
# x1 : stretch parameter, + means broader lightcurve
# redshift : calculated from sncosmo fit
# t0 : time of maximum brightness in B-band
# x0 : overall flux scaling parameter
# x1 : stretch parameter, + means broader lightcurve

Loaded 9 object(s) from /Users/david/Code/msc/runs/test2/sncosmo_fits.csv


,ztf_id,redshift,ncall,ndof,chisq,t0,x0,x1,c
0,ZTF19aaagfme,0.055000,114,53,218.135364,59775.687404,0.000919,1.317302,0.094216
1,ZTF19aaairqh,0.022000,125,26,91.040120,58499.181571,0.002330,-0.081931,0.348817
2,ZTF19aaarhtg,0.028000,171,21,229.736749,59048.243524,0.000411,-3.552564,0.647035
3,ZTF19aabmybj,0.040000,111,20,369.399294,58503.282769,0.002483,0.152742,0.024349
4,ZTF19aabvfwn,0.041382,105,5,5.348987,58501.911025,0.001590,-1.461401,-0.032020


In [15]:
# Nuisance parameters
alpha = 0.14
beta = 3.1
M = -19.36
cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

# build model once (faster), then set params each loop
model = sncosmo.Model(source="salt3")

ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)
processed_rows = []

for idx, row in df.iterrows():
    ztf_id = row['ztf_id']

    try:
        z = ztf_df.loc[ztf_df['ZTFID'] == ztf_id, 'host_redshift'].iloc[0]
    except IndexError:
        z = row['redshift']

    if pd.isna(z) or z == '' or (isinstance(z, str) and not str(z).strip()):
        z = row['redshift']
    z = float(z)

    ncall = row['ncall']
    ndof = row['ndof']
    chisq = row['chisq']
    t0 = float(row['t0'])
    x0 = float(row['x0'])
    x1 = float(row['x1'])
    c  = float(row['c'])

    # --- SALT3-safe mB ---
    model.set(z=z, t0=t0, x0=x0, x1=x1, c=c)
    mB = model.bandmag("bessellb", "ab", t0)   # rest-frame B at peak

    mu_obs = mB - M + alpha * x1 - beta * c
    mu_th  = cosmo.distmod(z).value
    resid  = mu_obs - mu_th

    processed_rows.append({
        'ztf_id': ztf_id,
        'host_redshift': z,
        'ncall': ncall,
        'ndof': ndof,
        'chisq': chisq,
        't0': t0,
        'x0': x0,
        'x1': x1,
        'c': c,
        'mB': mB,
        'mu_obs': mu_obs,
        'mu_th': mu_th,
        'resid': resid,
    })

    print(f"{ztf_id}: z={z:.5f}, mB={mB:.3f}, mu_obs={mu_obs:.3f}, mu_th={mu_th:.3f}, resid={resid:.3f}")

output_df = pd.DataFrame(processed_rows)
output_path = run_folder / "distance_process.csv"
output_df.to_csv(output_path, index=False)
print(f"\nWrote processed results to {output_path}")


ZTF19aaagfme: z=0.05500, mB=18.132, mu_obs=37.385, mu_th=36.949, resid=0.435
ZTF19aaairqh: z=0.02200, mB=17.111, mu_obs=35.378, mu_th=34.907, resid=0.471
ZTF19aaarhtg: z=0.02800, mB=19.024, mu_obs=35.881, mu_th=35.441, resid=0.441
ZTF19aabmybj: z=0.04000, mB=17.033, mu_obs=36.339, mu_th=36.234, resid=0.105
ZTF19aabvfwn: z=0.04138, mB=17.518, mu_obs=36.773, mu_th=36.310, resid=0.463
ZTF19aabyheu: z=0.02100, mB=16.286, mu_obs=35.134, mu_th=34.805, resid=0.330
ZTF19aabyppp: z=0.03700, mB=16.944, mu_obs=35.680, mu_th=36.060, resid=-0.381
ZTF19aabyuze: z=0.04300, mB=16.752, mu_obs=36.198, mu_th=36.396, resid=-0.198
ZTF19aacgslb: z=0.00452, mB=13.282, mu_obs=32.546, mu_th=31.442, resid=1.104

Wrote processed results to /Users/david/Code/msc/runs/test2/distance_process.csv
